### Ingest results.json file

##### Read the json file using spark df reader

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, FloatType, TimestampType
from pyspark.sql.functions import col, current_timestamp, lit, to_timestamp,concat

In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), True),
                                     StructField("raceId", IntegerType(), True),
                                     StructField("driverId", IntegerType(), True),
                                     StructField("constructorId", IntegerType(), True),
                                     StructField("number", IntegerType(), True),
                                     StructField("grid", IntegerType(), True),
                                     StructField("position", IntegerType(), True),
                                     StructField("positionText", StringType(), True),
                                     StructField("positionOrder", IntegerType(), True),
                                     StructField("points", FloatType(), True),
                                     StructField("lap", IntegerType(), True),
                                     StructField("time", StringType(), True),
                                     StructField("milliseconds", IntegerType(), True),
                                     StructField("fastestLap", IntegerType(), True),
                                     StructField("rank", IntegerType(), True),
                                     StructField("fastestLapTime", StringType(), True),
                                     StructField("fastestLapSpeed", StringType(), True),
                                     StructField("statusId", IntegerType(), True)])

In [0]:
results_df = spark.read \
.option("header", "true") \
.schema(results_schema) \
.json(f"{raw_folder_path}/results.json")

##### Select the columns

In [0]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                     .withColumnRenamed("raceId", "race_id") \
                                     .withColumnRenamed("driverId", "driver_id") \
                                     .withColumnRenamed("constructorId", "constructor_id") \
                                     .withColumnRenamed("positionText", "position_text") \
                                     .withColumnRenamed("positionOrder", "position_order") \
                                     .withColumnRenamed("fastestLap", "fastest_lap") \
                                     .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                     .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                     .withColumn("ingestion_date", current_timestamp()) \
                                     .withColumn("data_source", lit(v_data_source))

In [0]:
results_final_df = results_with_columns_df.drop("statusId")
                                          

Write the result as parquet file

In [0]:
#results_final_df.write.mode("overwrite").partitionBy("race_id").parquet(f"{processed_folder_path}/results")

In [0]:
results_final_df.write.mode("overwrite").partitionBy("race_id").format("parquet").saveAsTable("f1_processed.results")

In [0]:
#%fs
#ls /mnt/formula1dlstudy/processed/results

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/results"))

In [0]:
%sql
SELECT * FROM f1_processed.results

In [0]:
dbutils.notebook.exit("Success")